In [4]:
# 慢版

# import re
# import requests
# from huggingface_hub import hf_hub_download
# from llama_cpp import Llama
# from googletrans import Translator
# import torch
# from PIL import Image
# import matplotlib.pyplot as plt
# import io
# from TTS.api import TTS

# """Text-Generation > story_info"""

# def text_generation(title):
#     split_words = ['paragraph 1:', 'illustration 1:', 'paragraph 2:', 'illustration 2:', 'paragraph 3:', 'illustration 3:', 'paragraph 4:', 'illustration 4:']
#     model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
#     model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"
#     model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)
#     lcpp_llm = Llama(
#         model_path=model_path,
#         n_threads=2,
#         n_batch=512,
#         n_gpu_layers=32
#     )
#     lcpp_llm.params.n_gpu_layers
#     prompt = f'''
#     Taking Andersen's story as an example, write a 'very short' picture book story. It should includes four paragraphs (no more than 30 words) and illustration description in each paragraph.
#     Title: {title}.
#     Format please follows:
#         Title:
#         Paragraph 1:
#         Illustration 1:
#         Paragraph 2:
#         Illustration 2:
#         Paragraph 3:
#         Illustration 3:
#         Paragraph 4:
#         Illustration 4:
#     '''
#     while True:
#         print('RUN THE MODEL!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
#         response = lcpp_llm(prompt=prompt, max_tokens=1200, temperature=0.5, top_p=0.95, repeat_penalty=1.2, top_k=150, echo=True)
#         full_story = response["choices"][0]["text"]
#         if all(full_story.lower().count(word.lower()) == 2 for word in split_words):
#             print(full_story)
#             return full_story
        
# title = '小紅帽'
# full_story = text_generation(title)
# print(full_story)


In [5]:
# 測試版(加速版)

import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
# from peft import LoraConfig
# from trl import SFTTrainer

import torch
import gc

def clear_memory():
    torch.cuda.empty_cache()
    gc.collect()

# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model
new_model = "llama-2-7b-chat-guanaco"

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)


model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

def text_generation(title):
    prompt = f'''
        Taking Andersen's story as an example, write a 'very short' picture book story in English. 
        Title: {title}.
        It should includes four paragraphs, please follow the output format:
            Title:
            Paragraph 1:
            Illustration 1:
            Paragraph 2:
            Illustration 2:
            Paragraph 3:
            Illustration 3:
            Paragraph 4:
            Illustration 4:
        '''
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700,truncation=True)
    result = pipe(prompt)
    full_story = result[0]['generated_text']
    full_story = full_story.replace(prompt,'')
    
    clear_memory()  # Clear memory after generation
    return full_story

def extract_story_info(text):
    import re
    split_words = ['paragraph 1:', 'illustration 1:', 'paragraph 2:', 'illustration 2:', 'paragraph 3:', 'illustration 3:', 'paragraph 4:', 'illustration 4:']
    keys = ['paragraph 1', 'illustration 1', 'paragraph 2', 'illustration 2', 'paragraph 3', 'illustration 3', 'paragraph 4', 'illustration 4']

    # Split text into required paragrapgs and illustrations
    pattern = '|'.join(map(re.escape, split_words)) # Construct a regular expression (正則表達式) pattern to match multiple keywords and ignore case (大小寫)
    pattern = f'(?i)({pattern})'
    split_text = re.split(pattern, text) # Split using regular expressions
    split_text = [chunk.strip() for chunk in split_text if chunk.strip()] # Remove empty strings

    # Build dictionary
    story_info = {}
    for i in range(len(keys)):
        key = keys[i]
        value = split_text[i * 2 + 11] # 有...就要用18
        story_info[key] = value

    # Delete some ending sentence like 'I hope this helps you to write a delightful story for young readers!' in the end
    if '\n' in story_info['illustration 4']:
        split_illustration = story_info['illustration 4'].split('\n')
        story_info['illustration 4'] = '\n'.join(split_illustration[:-1])

    return story_info

def translate_to_eng(title):
    # !pip install googletrans==4.0.0rc1
    from googletrans import Translator, LANGUAGES

    # Generate translation
    translator = Translator()
    translation = translator.translate(title, dest='en')
    return translation.text

def generate_story(title):
    title = translate_to_eng(title)
    full_story = text_generation(title)
    story_info = extract_story_info(full_story)
    story_info['title'] = title
    return story_info


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
title = 'Little cat and dog'
full_story = text_generation(title)
print(full_story)



        Note: The story should be no more than 100 words.

Title: Little cat and dog

Paragraph 1:
Little cat and dog were the best of friends.

Illustration 1: A picture of a cat and dog playing together.

Paragraph 2:
They would play chase and cuddle, and have so much fun.

Illustration 2: A picture of the cat and dog running and jumping.

Paragraph 3:
One day, they found a ball and played with it all day.

Illustration 3: A picture of the cat and dog playing with a ball.

Paragraph 4:
They were so happy, and their friendship grew even stronger.

Illustration 4: A picture of the cat and dog sitting together, smiling.

Note: The story is very short, only 4 paragraphs, and each paragraph is around 20-25 words. The illustrations are simple and easy to understand.


In [8]:
story_info = generate_story(title)

# Show results
for key, value in story_info.items():
    print(f"{key}: {value}")
    print("=================================")
print(f'story_info = {story_info}')

ModuleNotFoundError: No module named 'googletrans'

In [9]:
!pip show googletrans


/bin/bash: /home/webapp/genai/bin/pip: /home/webapp/AI_PictureBooks_Web/genai/bin/python3: bad interpreter: No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
